<a href="https://colab.research.google.com/github/ollayyy/Popularity-Predictor/blob/main/Linear_regression_popularity_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs and imports

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=1296cce40eb714cfe833165cae5ea11883234412143fe689da1a52094a00f291
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
pip install findspark

In [3]:
import findspark
findspark.init()
import pandas as pd
import tensorflow as tf

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Call in data

In [6]:
spark = SparkSession.builder.appName("PopularityPredictor").getOrCreate()

In [7]:
csv_path = "/content/drive/MyDrive/Data Analysis Bootcamp/spotify_data.csv"
df = spark.read.csv(csv_path, header=True, quote= "\"", escape="\"")

In [8]:
df.show()

+---+--------------------+--------------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|_c0|         artist_name|          track_name|            track_id|popularity|year|   genre|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|duration_ms|time_signature|
+---+--------------------+--------------------+--------------------+----------+----+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+-----------+--------------+
|  0|          Jason Mraz|     I Won't Give Up|53QF56cjZA9RTuuMZ...|        68|2012|acoustic|       0.483| 0.303|  4| -10.058|   1|     0.0429|       0.694|             0.0|   0.115|  0.139|133.406|     240166|             3|
|  1|          Jason Mraz|    93 Million Miles|1s8tP3jP4GZcyHDsj...|        50|2012|acoustic|   

Clean Data Columns

In [9]:
cols = ['popularity','danceability', 'time_signature', 'energy', 'tempo', 'key', 'speechiness', 'acousticness', 'liveness', 'valence', ]
cleaned_df = df.select(cols)
cleaned_df.show()

+----------+------------+--------------+------+-------+---+-----------+------------+--------+-------+
|popularity|danceability|time_signature|energy|  tempo|key|speechiness|acousticness|liveness|valence|
+----------+------------+--------------+------+-------+---+-----------+------------+--------+-------+
|        68|       0.483|             3| 0.303|133.406|  4|     0.0429|       0.694|   0.115|  0.139|
|        50|       0.572|             4| 0.454|140.182|  3|     0.0258|       0.477|  0.0974|  0.515|
|        57|       0.409|             4| 0.234|139.832|  3|     0.0323|       0.338|  0.0895|  0.145|
|        58|       0.392|             4| 0.251|204.961| 10|     0.0363|       0.807|  0.0797|  0.508|
|        54|        0.43|             4| 0.791|171.864|  6|     0.0302|      0.0726|    0.11|  0.217|
|        48|       0.566|             4|  0.57| 83.403|  2|     0.0329|       0.688|  0.0943|   0.96|
|        48|       0.575|             4| 0.606|121.083|  9|       0.03|      0.011

In [10]:
clean_df=cleaned_df.toPandas()

In [11]:
df = pd.DataFrame(clean_df)
df = df.astype({"popularity":"int", "danceability": "float", "time_signature":"int",
                "energy":"float", "tempo":"float", "key":"int", "speechiness":"float", "acousticness":"float",
                "liveness":"float", "valence": "float"})

Machine Learning

In [12]:
df.dtypes

popularity          int64
danceability      float64
time_signature      int64
energy            float64
tempo             float64
key                 int64
speechiness       float64
acousticness      float64
liveness          float64
valence           float64
dtype: object

In [13]:
df=pd.get_dummies(df)

In [14]:
y=df['popularity'].values

In [15]:
x=df.drop('popularity',1).values

<ipython-input-15-300b452bd692>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x=df.drop('popularity',1).values


In [16]:
popularity_counts = df['popularity'].value_counts()
popularity_counts

0      158391
1       39820
2       31247
8       30322
9       29249
        ...  
93          3
94          1
96          1
100         1
95          1
Name: popularity, Length: 98, dtype: int64

In [17]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace= list(popularity_counts[popularity_counts<25000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    df['popularity'] = df['popularity'].replace(cls,"Other")
# Check to make sure binning was successful
df['popularity'].value_counts()

Other    547760
0        158391
1         39820
2         31247
8         30322
9         29249
7         28697
10        28600
3         28592
11        28325
6         27628
12        27297
13        26301
4         25929
14        25683
15        25466
5         25289
16        25168
Name: popularity, dtype: int64

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [20]:
number_input_features = len(x_train[0])
hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

nn.add(tf.keras.layers.Dense(units=1))

# Linear activation for regression
nn.compile(loss="mean_absolute_error", optimizer="adam")

fit_model = nn.fit(
    x_train_scaled,
    y_train,
    epochs=100
)

Epoch 1/100
28995/28995 [==============================] - 128s 4ms/step - loss: 12.2250
Epoch 2/100
28995/28995 [==============================] - 111s 4ms/step - loss: 12.0628
Epoch 3/100
28995/28995 [==============================] - 109s 4ms/step - loss: 12.0213
Epoch 4/100
28995/28995 [==============================] - 109s 4ms/step - loss: 11.9930
Epoch 5/100
28995/28995 [==============================] - 109s 4ms/step - loss: 11.9721
Epoch 6/100
28995/28995 [==============================] - 107s 4ms/step - loss: 11.9568
Epoch 7/100
28995/28995 [==============================] - 106s 4ms/step - loss: 11.9464
Epoch 8/100
28995/28995 [==============================] - 109s 4ms/step - loss: 11.9358
Epoch 9/100
28995/28995 [==============================] - 108s 4ms/step - loss: 11.9267
Epoch 10/100
28995/28995 [==============================] - 110s 4ms/step - loss: 11.9184
Epoch 11/100
28995/28995 [==============================] - 107s 4ms/step - loss: 11.9129
Epoch 12/100
28995/

In [23]:
# Evaluate the model using the test data
model_loss = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}")

7249/7249 - 17s - loss: 11.8849 - 17s/epoch - 2ms/step
Loss: 11.884917259216309
